# Unilever

### Automatización aplicación pagos

In [1]:
# 1. Imports y configuración

import pandas as pd
import numpy as np

In [2]:
import pandas as pd

def cargar_tabla_excel(path):
    """
    Carga la única tabla que hay en la primera hoja del Excel,
    detectando automáticamente dónde empieza.
    Funciona aunque los encabezados sean distintos para cada cliente.
    """
    # Leemos toda la primera hoja sin encabezados
    df = pd.read_excel(path, header=None)
    
    # Detectamos la primera fila que parece tabla:
    # condición: la fila no tiene celdas vacías y tiene varias columnas con datos
    for i, fila in df.iterrows():
        num_celdas_no_vacias = fila.count()
        if num_celdas_no_vacias >= 5:  # ajusta este número según mínimo de columnas que esperas
            df.columns = fila  # asignamos esa fila como encabezado
            df = df.iloc[i+1:].reset_index(drop=True)  # descartamos filas anteriores y reseteamos índice
            break

    return df

In [54]:
import pandas as pd

def cargar_remittance(path, sheet_name):
    """
    Carga la tabla de remittance detectando automáticamente
    la fila donde empiezan los encabezados.
    """
    # Leemos todo sin asignar headers
    df = pd.read_excel(path, sheet_name=sheet_name, header=None)
    
    # Lista de columnas esperadas
    columnas_esperadas = [
        "COD DOC", "DOC", "No. Interno", "No. Doc", "Fecha Doc", 
        "Cod Ean Sitio Entrega", "Lugar Entrega", "Ref. Pedido", 
        "Ref. Factura", "Ref. Avis Recibo", "Ref. Avis Devol", 
        "Ref. Dossier", "Ref. Numero de Folio", "Total a Pagar", 
        "Vlr Ant. Ajuste", "Ajustes", "Iva", "Impuestos", 
        "Descuentos", "Vlr Hort", "Valor Doc", "Rete fte", 
        "Rete Iva", "Rete Ica", "Rte. CREE", "Cod Ajustes", 
        "Descripcion Ajustes", "Valor Ajustado", "Observaciones", 
        "No Lin", "Código de identificación del Item", 
        "Tipo de Identificación", "Descripción del Item", 
        "Precio Neto Unitario del Item", "Razón del Ajuste1", 
        "Cantidad del Item para el Ajuste1", "Razón del Ajuste2", 
        "Cantidad del Item para el Ajuste2", "Razón del Ajuste3", 
        "Cantidad del Item para el Ajuste3"
    ]
    
    # Buscar la fila que coincide con los encabezados
    for i, fila in df.iterrows():
        if all(col in fila.values for col in columnas_esperadas[:5]):  # se puede usar solo primeras columnas
            df.columns = fila  # asignamos encabezados
            df = df.iloc[i+1:]  # descartamos filas anteriores
            break
    
    # Seleccionamos solo las columnas que nos interesan
    columnas_relevantes = ["DOC", "No. Doc", "Total a Pagar"]
    df = df[columnas_relevantes].copy()
    
    return df


In [3]:
# ------------------------------------------
# Función para cargar los datos del Excel
# ------------------------------------------
def cargar_remittance(path, sheet_name):
    """
    Carga la hoja de remittance desde el archivo Excel
    y selecciona solo las columnas relevantes.
    """
    df = pd.read_excel(
        path,
        sheet_name=sheet_name,
        skiprows=25,           # fila 26 será la cabecera
        nrows=48,              # filas de la 26 a la 73
        usecols=["DOC", "No. Doc", "Total a Pagar"]  # columnas necesarias
    )
    return df

In [4]:
# ------------------------------------------
# Función para limpiar y transformar los datos
# ------------------------------------------
def limpiar_remittance(df):
    """
    - Renombra columnas para mayor claridad
    - Filtra solo los documentos de tipo 'Factura Comercial'
    - Ajusta la columna 'Referencia / Factura' eliminando primer carácter y últimos 3
    - Reemplaza el nombre de documento por 'Factura'
    """
    df = (
        df.rename(columns={
            "DOC": "Tipo de Documento",
            "No. Doc": "Referencia / Factura"
        })
        # Filtra solo Facturas Comerciales
        .loc[lambda x: x["Tipo de Documento"] == "Factura Comercial"]
        # Ajusta la columna 'Referencia / Factura'
        .assign(
            **{
                "Referencia / Factura": lambda x: x["Referencia / Factura"].str[1:-3],
                "Tipo de Documento": lambda x: x["Tipo de Documento"].replace("Factura Comercial", "Factura")
            }
        )
    )
    return df

In [5]:
# ------------------------------------------
# Función principal
# ------------------------------------------
def main():
    # Cargar datos
    path = "Remittance.xlsx"
    sheet_name = "REMADV202508120820_1182089"
    remittance = cargar_remittance(path, sheet_name)
    
    # Limpiar y transformar
    remittance = limpiar_remittance(remittance)
    
    # Mostrar resultados
    print(remittance.head())

In [6]:
# ------------------------------------------
# Punto de entrada del script
# ------------------------------------------
if __name__ == "__main__":
    main()

  Tipo de Documento Referencia / Factura  Total a Pagar
0           Factura           PMP1243434      165989699
1           Factura           PMP1243435       29856410
2           Factura           PMP1243436       12528130
3           Factura           PMP1243437          83124
4           Factura           PMP1243438       53514044
